In [ ]:
import numpy as np
import pandas as pd

from datetime import date
from datetime import datetime

from sklearn.neighbors import KernelDensity

import seaborn as sns
import random as r

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
# For downsampling
from sklearn.utils import resample,shuffle
np.random.seed(2021)

In [ ]:
df_companies = pd.read_csv("./companies.csv")
df_payments = pd.read_csv("./payments.csv")
df_physicians = pd.read_csv("./physicians.csv")

In [ ]:
df_companies.rename({"State":"State_Comp","Country":"Country_Comp"}, axis='columns', inplace=True)
df_physicians.rename({"State":"State_Phys","Country":"Country_Phys"}, axis='columns', inplace=True)
pd.set_option('display.max_columns', None)
df = pd.merge(df_payments, df_physicians, left_on='Physician_ID', right_on='id', how='left').drop('id', axis=1)
df = pd.merge(df, df_companies, on='Company_ID', how='left')
df.rename({"Name":"Company_Name"}, axis='columns', inplace=True)

###test safekeeping dataframe
df_main_test=df[df.set=="test"]



In [ ]:
#lower case all strings
df = df.astype(str)
for column in df.columns:
     if(df[column].dtype=='object'):
        df[column] = df[column].str.lower()

# Data Squeezing
df_squeezed=df
df_squeezed['index_col'] = df.index
df_squeezed=pd.wide_to_long(df_squeezed, stubnames='Product_Name_', i=['index_col'],j='Product_Name_Number').reset_index()
df_squeezed.drop(columns=['index_col'],inplace=True)

dfNew=df_squeezed.query('Product_Name_Number=="1"')
dfNew['Product_Type']=dfNew['Product_Type_1']
dfNew2=df_squeezed.query('Product_Name_Number=="2"')
dfNew2['Product_Type']=dfNew2['Product_Type_2']
dfNew3=df_squeezed.query('Product_Name_Number=="3"')
dfNew3['Product_Type']=dfNew3['Product_Type_3']
dfNew=pd.concat([dfNew, dfNew2])
dfNew=pd.concat([dfNew, dfNew3])

string="Code"
dfNew1=dfNew.query('Product_Name_Number=="1"')
dfNew1['Product_'+string]=dfNew1['Product_'+string+'_1']
dfNew2=dfNew.query('Product_Name_Number=="2"')
dfNew2['Product_'+string]=dfNew2['Product_'+string+'_2']
dfNew3=dfNew.query('Product_Name_Number=="3"')
dfNew3['Product_'+string]=dfNew3['Product_'+string+'_3']
dfNew1=pd.concat([dfNew1, dfNew2])
dfNew1=pd.concat([dfNew1, dfNew3])
dfNew=dfNew1
dfNew

string="Category"
dfNew1=dfNew.query('Product_Name_Number=="1"')
dfNew1['Product_'+string]=dfNew1['Product_'+string+'_1']
dfNew2=dfNew.query('Product_Name_Number=="2"')
dfNew2['Product_'+string]=dfNew2['Product_'+string+'_2']
dfNew3=dfNew.query('Product_Name_Number=="3"')
dfNew3['Product_'+string]=dfNew3['Product_'+string+'_3']
dfNew1=pd.concat([dfNew1, dfNew2])
dfNew1=pd.concat([dfNew1, dfNew3])
dfNew=dfNew1

dfNew['index_col'] = dfNew.index
dfNew=pd.wide_to_long(dfNew, stubnames='License_State_', i=['index_col'],j='License_State_Number').reset_index()
dfNew.drop(columns=['index_col'],inplace=True)

#Dropping Columns
dfNew.drop(columns=['Product_Name_Number','License_State_Number','First_Name', 'Middle_Name', 'Last_Name', 'Name_Suffix', 
                 'Company_Name','Province','Product_Category_1','Product_Category_2','Product_Category_3',
                 'Product_Type_1','Product_Type_2','Product_Type_3',
                'Product_Code_1','Product_Code_2','Product_Code_3'],inplace=True)

dfNewNONE=dfNew.where(dfNew!="nan",None)
dfScrap=dfNewNONE.dropna(subset=['Product_Name_'])
dfScrap=dfScrap.dropna(subset=['Product_Type'])
df=dfScrap

In [ ]:
#ENCODING CELL
df = df.astype(str)
for column in df.columns:
     if(df[column].dtype=='object'):
        df[column] = df[column].str.lower()

#Convert to Date
df['Date']=pd.to_datetime(df['Date'])

#type conv after str conv
df[['Company_ID','Number_of_Payments']]=df[['Company_ID','Number_of_Payments']].astype(int)
df['Total_Amount_of_Payment_USDollars']=df['Total_Amount_of_Payment_USDollars'].astype(float)


columns_encode=['Product_Code','Product_Name_','License_State_','Product_Type','Product_Category','Nature_of_Payment_or_Transfer_of_Value','Form_of_Payment_or_Transfer_of_Value','Zipcode','Third_Party_Recipient', 'Charity', 'Third_Party_Covered','Contextual_Information', 'Related_Product_Indicator','Primary_Specialty']
for column in columns_encode:
    df[column]=df[column].astype('category')
    df[column]=df[column].cat.codes
    
name_list = pd.unique(df[['Country_Comp', 'Country_Phys','Country_of_Travel']].values.ravel('K'))
# name_list = np.delete(name_list,np.where(name_list=="nan"))
name_list_dict = {}
int_code=int(1)
for name in name_list:
    name_list_dict.update({name:int_code})
    int_code+=1

def product_encode(product_name):
    return name_list_dict.get(product_name)

df['Country_Comp'] = df['Country_Comp'].apply(product_encode)
df['Country_Phys'] = df['Country_Phys'].apply(product_encode)
df['Country_of_Travel'] = df['Country_of_Travel'].apply(product_encode)

name_list = pd.unique(df[['State_Comp', 'State_Phys', 'State_of_Travel']].values.ravel('K'))
# name_list = np.delete(name_list,np.where(name_list=="nan"))
name_list_dict = {}
int_code=int(1)
for name in name_list:
    name_list_dict.update({name:int_code})
    int_code+=1

def product_encode(product_name):
    return name_list_dict.get(product_name)

df['State_Comp'] = df['State_Comp'].apply(product_encode)
df['State_Phys'] = df['State_Phys'].apply(product_encode)
df['State_of_Travel'] = df['State_of_Travel'].apply(product_encode)

name_list = pd.unique(df[['City_of_Travel', 'City']].values.ravel('K'))
# name_list = np.delete(name_list,np.where(name_list=="nan"))
name_list_dict = {}
int_code=int(1)
for name in name_list:
    name_list_dict.update({name:int_code})
    int_code+=1

def product_encode(product_name):
    return name_list_dict.get(product_name)

df['City_of_Travel'] = df['City_of_Travel'].apply(product_encode)
df['City'] = df['City'].apply(product_encode)

df['Ownership_Indicator']=np.where((df.Ownership_Indicator == "yes"),1,df.Ownership_Indicator)
df['Ownership_Indicator']=np.where((df.Ownership_Indicator == "no"),0,df.Ownership_Indicator)
df['Ownership_Indicator']=df['Ownership_Indicator'].astype(int)

df['Date_year']=df['Date'].dt.year
df['Date_month']=df['Date'].dt.month
df['Date_day']=df['Date'].dt.day
df.drop(columns=['Date'],inplace=True)

In [ ]:
###SPLIT
df_test=df[df.set=="test"]
df_train=df[df.set=="train"]

###drop set column
df_test.drop(columns=['set'],inplace=True)
df_train.drop(columns=['set'],inplace=True)

##send predicted attribute to end
first_col = df_train.pop("Ownership_Indicator")
df_train.insert((df_train.shape[1]),"Ownership_Indicator",first_col)

first_col = df_test.pop("Ownership_Indicator")
df_test.insert((df_test.shape[1]),"Ownership_Indicator",first_col)

##send Physician ID to end
first_col = df_train.pop("Physician_ID")
df_train.insert((df_train.shape[1]),"Physician_ID",first_col)

first_col = df_test.pop("Physician_ID")
df_test.insert((df_test.shape[1]),"Physician_ID",first_col)

##send Record ID to end
first_col = df_train.pop("Record_ID")
df_train.insert((df_train.shape[1]),"Record_ID",first_col)

first_col = df_test.pop("Record_ID")
df_test.insert((df_test.shape[1]),"Record_ID",first_col)

In [ ]:
np.random.seed(2021)
###downsampling
df_train_maj=df_train[df_train.Ownership_Indicator==0]
df_train_min=df_train[df_train.Ownership_Indicator==1]
df_train_maj_down=resample(df_train_maj, replace=False, n_samples=df_train_min.shape[0], random_state=2021)
df_train_final = pd.concat([df_train_maj_down, df_train_min])

###test train split of own train set
df_train_final['is_train']=np.random.uniform(0,1,len(df_train_final)) <=.80
train, test = df_train_final[df_train_final['is_train']==True], df_train_final[df_train_final['is_train']==False]
train = shuffle(train,random_state=2021)
test = shuffle(test,random_state=2021)

In [ ]:
###feature and label split
features=train[train.columns[0:(train.shape[1]-4)]]
y=train['Ownership_Indicator']
x_test = test[test.columns[0:(test.shape[1]-4)]]
y_test = test['Ownership_Indicator']

###model training
clf = RandomForestClassifier(n_jobs=-1, oob_score = True, n_estimators = 196, random_state=2021)
clf.fit(features, y)
test_acc=clf.score(x_test,y_test)

    # Accuracy Scores
print ('Internal Accuracy Score', clf.oob_score_)
print ('RF accuracy: TRAINING', clf.score(features,y))
print ('RF accuracy: TESTING', test_acc)

In [ ]:
phys_id_list=df_test['Physician_ID']
predict_set=df_test
predict_set.drop(columns=['Physician_ID','Ownership_Indicator','Record_ID'],inplace=True)

In [ ]:
score = clf.predict(predict_set)

In [ ]:
submit={'Physician_ID':phys_id_list, 'Score':score}
submit_df=pd.DataFrame(data=submit)
physUnique=submit_df['Physician_ID'].unique()

sc=[None]*1000
for i in range(physUnique.size):
    physician=physUnique[i]
    if(submit_df.query('Physician_ID==@physician and Score==1').shape[0]>0):
        sc[i]=1
    else:
        sc[i]=0

final_submit={'id':physUnique, 'prediction':sc}
fs_df=pd.DataFrame(data=final_submit)
fs_df.to_csv("./submission5.csv",index=False)